In [2]:
import pandas as pd
import defusedxml.ElementTree as ET
import re

## Školská zařízení (MŠ, ZŠ, střední aj.) - scrap kapacit

In [3]:
tree = ET.parse('vrejcelk.xml')
root = tree.getroot()

In [38]:
skoly_data = []
zarizeni_info_data = []
zarizeni_misto_data = []
skola_obor_data = []

for ps in root.findall('.//PravniSubjekt'):
    # TAB "skoly" a "zarizeni_info"
    ico = ps.find('ICO').text
    nazev = ps.find('Reditelstvi').find('RedPlnyNazev').text
    for sz in ps.find('SkolyZarizeni').findall('SkolaZarizeni'):
        izo = sz.find('IZO').text
        typZarizeni = sz.find('SkolaPlnyNazev').text
        zarizeniDruhTyp = sz.find('SkolaDruhTyp').text
        zarizeniKapacita = sz.find('SkolaKapacita').text
        kapacitaJednotka = sz.find('SkolaKapacitaJednotka').text
        jazyk = sz.find('SkolaJazyk').text

        raw_skoly = {'IZO' : izo,  
                    'ZarizeniTyp': typZarizeni, 
                    'Kapacita': zarizeniKapacita,
                    'KapacitaJednotka': kapacitaJednotka,           
                    }
        raw_zarizeni_info = {
                    'ICO': ico, 
                    'ReditelstviNazev' : nazev, 
                    'ZarizeniDruhTyp' : zarizeniDruhTyp, 
                    'jazyk' : jazyk, 
                    'IZO' : izo,
                    }
        skoly_data.append(raw_skoly)
        zarizeni_info_data.append(raw_zarizeni_info)

        # TAB "zarizeni_misto"
        for smvc in sz.find('SkolaMistaVykonuCinnosti').findall('SkolaMistoVykonuCinnosti'):
            druh = smvc.find('MistoDruhTyp').text # u středních škol např. rozlišení gymnázií, u MŠ specializace
            #RUAINKod = smvc.find('MistoRUAINKod').text
            #adresa1 = smvc.find('MistoAdresa1').text

            # extrahování PSČ
            def extract_psc(adresa_text):
                if adresa_text:
                    PSC_match = re.search(r'\d{3} ?\d{2}', adresa_text)
                    return PSC_match.group() if PSC_match else adresa_text
                return None
            PSC = extract_psc(smvc.find('MistoAdresa2').text)        
            PSC2 = extract_psc(smvc.find('MistoAdresa3').text)

            
                        
            raw_zarizeni_misto = {
                    'druh' : druh,
                    #'RUAINKod' : RUAINKod,
                    #'adresa1': adresa1,
                    'adresa2': PSC,
                    'adresa3': PSC2,
                    'IZO' : izo,
                   }
            zarizeni_misto_data.append(raw_zarizeni_misto)
            
        # TAB "skola_obor"
        if sz.find('SkolaOboryVzdelani') is not None:
            for sov in sz.find('SkolaOboryVzdelani').findall('SkolaOborVzdelani'):
                oborNazev = sov.find('OborNazev').text
                delkaVzdelavani = sov.find('DelkaVzdelavani').text
                oborKapacita = sov.find('OborKapacita').text
                oborDobihajici = sov.find('OborDobihajici').text
                # print(oborNazev)
    
                raw_skola_obor = {
                    'OborNazev' : oborNazev,
                    'DelkaVzdelavani': delkaVzdelavani,
                    'OborKapacita' : oborKapacita,
                    'OborDobihajici' : oborDobihajici,
                    'IZO' : izo,
                   }
                skola_obor_data.append(raw_skola_obor)

### Školy (kapacita) data -> dataframe dle ERD

Celkem 547 zařízení nemá vyplněný RUAINKod, -> znovuotevření řešení skrze PSČ, 1 PSČ může mít vícero krajů, 

In [89]:
skoly = pd.DataFrame(skoly_data)

zarizeni_info = pd.DataFrame(zarizeni_info_data)
zarizeni_info.insert(loc=0, column='ID', value=range(1, len(zarizeni_info_data) + 1))
    
zarizeni_misto = pd.DataFrame(zarizeni_misto_data)
zarizeni_misto.insert(loc=0, column='ID', value=range(1, len(zarizeni_misto) + 1))
    
skola_obor = pd.DataFrame(skola_obor_data)
skola_obor.insert(loc=0, column='ID', value=range(1, len(skola_obor_data) + 1))
  
def makePsc2(row):
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if adresa3:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
        if psc_match:
            return psc_match.group().replace(' ', '')
    elif adresa2:
        psc_match = re.search(r'\d{3} ?\d{2}', adresa2)
        if psc_match:
            return psc_match.group().replace(' ', '')
            
def makeObec(row):
    PSC_obec = row['PSC']
    adresa2 = row['adresa2']
    adresa3 = row['adresa3']
    if PSC_obec:
        if adresa3:
            psc_match = re.search(r'\d{3} ?\d{2}', adresa3)
            if psc_match.group().replace(' ', '') == PSC_obec:
                return adresa2
        return None
    else:
        return None
        

zarizeni_misto['PSC'] = zarizeni_misto.apply(
    makePsc2, axis=1) 

zarizeni_misto['obec'] = zarizeni_misto.apply(   
    makeObec, axis=1)
    
# null hodnoty nahrazeny číslem 0, následně celý sloupe převeden na int
zarizeni_misto['PSC'] = zarizeni_misto['PSC'].fillna(0)
zarizeni_misto = zarizeni_misto.astype({'PSC':'int'})
zarizeni_misto

,ID,druh,adresa2,adresa3,IZO,PSC,obec
0,1,Mateřská škola,Nové Město,110 00,049625918,11000,Nové Město
1,2,Školní jídelna,Nové Město,110 00,102413096,11000,Nové Město
2,3,Mateřská škola,Kamýk,142 00,107500884,14200,Kamýk
3,4,Školní jídelna - výdejna,Kamýk,142 00,161102263,14200,Kamýk
4,5,Mateřská škola pro děti se zdravotním postižením,Kamýk,142 00,110034384,14200,Kamýk
...,...,...,...,...,...,...,...
38528,38529,Základní škola,Modřany,143 00,181146631,14300,Modřany
38529,38530,Školní jídelna,Želivec,251 68,181146797,25168,Želivec
38530,38531,Lesní mateřská škola,290 01,None,181146878,29001,None
38531,38532,Lesní mateřská škola,251 64,None,181146967,25164,None


In [95]:
psc_okres = pd.read_csv(r'D:\git\skolstvi_projekt\adresa\psc2okres.csv') # obsahuje duplicity, 77 okresů
okres_kraj = pd.read_csv(r'D:\git\skolstvi_projekt\adresa\okres2kraj.csv') # pouze unikátní hodnoty, míň okresu, než má psc okresů, 82 okresů
psc_okres = psc_okres.drop_duplicates() # dropnutí duplikátu z cca 17tis na cca 3tis záznamů
psc_okres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3165 entries, 0 to 16951
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PSC     3165 non-null   int64 
 1   Okres   3165 non-null   object
dtypes: int64(1), object(1)
memory usage: 74.2+ KB


In [68]:
# vytvoreni nové tabulky -> dle PSČ zjistime pravděpodobně kraj
psc_kraj = okres_kraj.merge(psc_okres, how='right', on='Okres')
# len(psc_kraj['PSC'].unique()) #3007, 3165 je celkový počet, tzn. 158 PSČ celkem figuruje ve vícero krajích

zarizeni_misto2 = zarizeni_misto.merge(psc_kraj, how='left', on='PSC')
zarizeni_misto2
# zarizeni_misto2[(zarizeni_misto2['Kraj'].isnull()) & (zarizeni_misto2['Okres'].isnull())]['druh'].value_counts()



<class 'pandas.core.frame.DataFrame'>
Index: 3165 entries, 0 to 16951
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   PSC     3165 non-null   object
 1   Okres   3165 non-null   object
dtypes: object(2)
memory usage: 74.2+ KB


,ID,druh,adresa2,adresa3,IZO,PSC,obec,Okres,Kraj
0,1,Mateřská škola,Nové Město,110 00,049625918,110 00,Nové Město,Hlavní město Praha,NaN
1,2,Školní jídelna,Nové Město,110 00,102413096,110 00,Nové Město,Hlavní město Praha,NaN
2,3,Mateřská škola,Kamýk,142 00,107500884,142 00,Kamýk,Hlavní město Praha,NaN
3,4,Školní jídelna - výdejna,Kamýk,142 00,161102263,142 00,Kamýk,Hlavní město Praha,NaN
4,5,Mateřská škola pro děti se zdravotním postižením,Kamýk,142 00,110034384,142 00,Kamýk,Hlavní město Praha,NaN
...,...,...,...,...,...,...,...,...,...
42006,38530,Školní jídelna,Želivec,251 68,181146797,251 68,Želivec,Benešov,Středočeský kraj
42007,38530,Školní jídelna,Želivec,251 68,181146797,251 68,Želivec,Praha-východ,Středočeský kraj
42008,38531,Lesní mateřská škola,290 01,None,181146878,290 01,None,Nymburk,Středočeský kraj
42009,38532,Lesní mateřská škola,251 64,None,181146967,251 64,None,Praha-východ,Středočeský kraj


In [7]:
# zarizeni_misto2[zarizeni_misto2['Kraj'].isnull()]['Okres'].info() #non null 3357
# zarizeni_misto2[zarizeni_misto2['Kraj'].isnull()]#['Okres'] # 3518 rows
# zarizeni_misto2.info()
# zarizeni_misto2
# zarizeni_misto2[['Okres', 'Kraj']]
zarizeni_misto2[(zarizeni_misto2['druh'] == 'Mateřská škola') & (zarizeni_misto2['Okres'].isnull())]
# zarizeni_misto2[zarizeni_misto2['druh'] == 'Mateřská škola']['Okres'].info()


# NUTNO POŘEŠIT PSČ VS 2 KRAJE!


,ID,druh,adresa2,adresa3,IZO,PSC,Okres,Kraj
16445,14930,Mateřská škola,330 40,None,115600621,330 40,NaN,NaN
28004,25579,Mateřská škola,793 64,None,107620677,793 64,NaN,NaN
28018,25593,Mateřská škola,793 43,None,181004011,793 43,NaN,NaN
29034,26482,Mateřská škola,Louky,735 73,107623731,735 73,NaN,NaN
29622,27020,Mateřská škola,774 00,None,119901269,774 00,NaN,NaN
36279,33298,Mateřská škola,150 30,None,162104316,150 30,NaN,NaN
38340,35152,Mateřská škola,Dejvice,160 41,181026899,160 41,NaN,NaN
38341,35153,Mateřská škola,Žižkov,130 67,181026899,130 67,NaN,NaN


## DS, MŠ + PSZŠSpeciální

### MŠ + PSZŠSpeciální (kapacita) TAB

In [8]:
materske_skoly = skoly[skoly['ZarizeniTyp'].isin(["Mateřská škola", "Mateřská škola (lesní mateřská škola)", "Mateřská škola - Przedszkole", "Mateřská škola s p.j.v.", "Přípravný stupeň základní školy speciální"])]
materske_skoly = materske_skoly.astype({'Kapacita':'int'})
# materske_skoly.info()
# materske_skoly['Kapacita'].sum()
materske_skoly

,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka
0,049625918,Mateřská škola,54,dítě
2,107500884,Mateřská škola,70,dítě
4,110034384,Mateřská škola,6,dítě
6,060437171,Mateřská škola,130,dítě
8,110035585,Mateřská škola,18,dítě
...,...,...,...,...
27171,181145014,Mateřská škola (lesní mateřská škola),15,dítě
27177,181145499,Mateřská škola (lesní mateřská škola),15,dítě
27181,181145871,Mateřská škola,44,dítě
27187,181146878,Mateřská škola (lesní mateřská škola),16,dítě


### DětskéSkupinky (realtime kapacita) TAB

In [9]:
from web_scrap_DS import DS_tabulka

In [10]:
DS_tabulka = DS_tabulka.astype({'Kapacita':'int'})
# DS_tabulka.info()
DS_tabulka

,ZarizeniTyp,Kapacita,Kraj
0,Dětská skupinka,4531,Hlavní město Praha
1,Dětská skupinka,1311,Jihočeský
2,Dětská skupinka,4058,Jihomoravský
3,Dětská skupinka,256,Karlovarský
4,Dětská skupinka,1137,Královéhradecký
5,Dětská skupinka,758,Liberecký
6,Dětská skupinka,2182,Moravskoslezský
7,Dětská skupinka,1386,Olomoucký
8,Dětská skupinka,1068,Pardubický
9,Dětská skupinka,1077,Plzeňský


### **Předškolní vzdělávání a péče TAB (kapacita a realtime kapacita)**

In [11]:
predskolniVP = pd.concat([materske_skoly, DS_tabulka], ignore_index=True, sort=False)
predskolniVP.info()
predskolniVP


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5501 entries, 0 to 5500
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   IZO               5487 non-null   object
 1   ZarizeniTyp       5501 non-null   object
 2   Kapacita          5501 non-null   int64 
 3   KapacitaJednotka  5482 non-null   object
 4   Kraj              14 non-null     object
dtypes: int64(1), object(4)
memory usage: 215.0+ KB


,IZO,ZarizeniTyp,Kapacita,KapacitaJednotka,Kraj
0,049625918,Mateřská škola,54,dítě,NaN
1,107500884,Mateřská škola,70,dítě,NaN
2,110034384,Mateřská škola,6,dítě,NaN
3,060437171,Mateřská škola,130,dítě,NaN
4,110035585,Mateřská škola,18,dítě,NaN
...,...,...,...,...,...
5496,NaN,Dětská skupinka,1077,NaN,Plzeňský
5497,NaN,Dětská skupinka,4052,NaN,Středočeský
5498,NaN,Dětská skupinka,1559,NaN,Ústecký
5499,NaN,Dětská skupinka,579,NaN,Vysočina


In [12]:
predskolniVP.to_csv('predskolniVP.csv', mode='w')

## Porodnost 2018-2023 TAB (neměnná data z webu ČSÚ)

Pro přesnost bychom od porodnosti měli správně odečítat počet zemřelých. Vzhledem k nedostatečně podrobným datům s tímto údajem pracovat nebudeme, jedná se ale o nižší stovky ve věku 0 - 14 let., podrobnější členění lze nalézst do 1roku, případně od 0-5let, což je pro nás stále nedostatečně podrobné.

Vzhledem k málo podrobným datům i v rámci porodnosti (roční souhr za každý kraj nám výsledek dostupnosti výchovných a školských zařízení zkreslí 8měsíců). Je nutno si tedy manuálně vytáhnout tabulku porodnosti z VD ČSU a následně si data, co nejpřesněji dopočítat. Prakticky se nejpodrobnější data dají získat a dopočítat za jednotlivá čtvrtletí, pro naše účely proto:

**(leden až září (n) - 1.pololetí (n)) / 3 = 1/3 3.čtvrtletí (n)**

**porodnost do začátku školního roku = 1.pololetí(n) + 2/3 3.čtvrtletí (n)**

**porodnost od začátku školního roku, která se přičte k 8.měsícům porodnosti roku n+1 -> (1/3 3.čtvrtletí + rok (n) - leden až září (n)) + 1.pololetí(n+1) + 2/3 3.čtvrtletí (n+1)** -> od dovršení 3 let do 31.8.x (respektive 31.8.(n+3) vzniká nárok nastoupit do školky, proto by tomu měli odpovídat regionální kapacity MŠ

V počtech dětí zaokrouhluejme na celá čísla nahoru.\
\
Porodnost (živě narozených) zahrnuje i narození dětí s jiným občanstvím. Prozatím se nepodařilo najít dostatečně podrobná data(porovnání krajů) na to, tento údaj zahrnout do dat. Lze dohledat počet "Živě narozených cizinců", ovšem jen k celé ČR. Od roku 1995 se podíl cizinců na živě narozených v ČR (v %) postupně zvyšuje z 0,7 na 6,4 (2022), na 8,3 (2023).
více infa: https://csu.gov.cz/pocet-cizincu-demograficke-udalosti?pocet=10&start=0&podskupiny=291&razeni=-datumVydani


### **Porodnost podrobná pro školní roky 2019/2020-2023/2024 dle regionů**

In [13]:
porodnost_podr = (pd.read_excel(r'D:\git\skolstvi_projekt\porodnost_podr_uprav.xlsx', sheet_name=0, header=6, index_col=0, usecols='B, AO:AS', nrows=14))
porodnost_podr


,porodnost do 2023/2024,porodnost do 2022/2023,porodnost do 2021/2022,porodnost do 2020/2021,porodnost do 2019/2020
Hlavní město Praha,12869,14178,14967,14819,15147
Jihočeský kraj,5559,6335,6709,6479,6752
Jihomoravský kraj,11133,12519,12990,12935,13489
Karlovarský kraj,2216,2436,2678,2742,2775
Kraj Vysočina,4615,5192,5478,5416,5462
Královéhradecký kraj,4770,5175,5499,5567,5586
Liberecký kraj,3703,4103,4442,4557,4695
Moravskoslezský kraj,9975,11243,11910,11871,12103
Olomoucký kraj,5523,6165,6456,6538,6481
Pardubický kraj,4671,5180,5399,5543,5609


### Porodnost tab, málo podrobná, prozatím odložená

In [14]:
# from web_scrap_porodnost import porodnost_tabulka

In [15]:
# porodnost_tab = porodnost_tabulka.astype({"PocetNarozenychDeti":'int'}) 
# porodnost_tabCR = porodnost_tab[porodnost_tab['Kraj'] == 'Česká republika']
# porodnost_tab_kraje = porodnost_tab[porodnost_tab['Kraj'] != 'Česká republika']
# # porodnost_tab_kraje
# # porodnost_tabCR[porodnost_tabCR['Rok'].isin([2021, 2020, 2019, 2018])]['PocetNarozenychDeti'].sum()
# # porodnost_tab_kraje.info()
# porodnost_tab_kraje

In [16]:
# porodnost_tab_kraje.to_csv('porodnost.csv', mode='w')

## ČSÚ: Školy a školská zařízení - reálný počet žáků

### Pro rozbor ZŠ, Do 1třídy nastupuje více dětí, než kolik mělo odklad. Vzhledem k povinnému poslednímu ročníku v MŠ se pravděpodobně jedná o imigraci?

In [17]:
# csu = pd.read_excel(r'D:\git\skolstvi_projekt\CSU\230042230202.xlsx', sheet_name=['2.2.17', '2.2.18'], header = 3, index_col=0, usecols='A, H:L')
# # 2.2.17 -> počet žáků nově přijatých do 1. ročníku celkem
# # 2.2.18 -> počet žáků 7letých a starších nově přijatých do 1. ročníku, podmnožina 2.2.17
# # pro pozdější užití, na kartě 2.2.20 -> žáci opakující ročník, údaj "z toho v 1.ročníku" nutno PŘIČÍST k údaji z 2.2.17, 
# #žel tento údaj nění formátovaný v časocé řadě, nutno pro každý ročník explicitně stáhnout samostatný xlsx soubor

# nPrijatiZS = pd.DataFrame(csu['2.2.17'])
# nPrijatiZS_7plus = pd.DataFrame(csu['2.2.18'])
# # nPrijatiZS
# # nPrijatiZS_7plus

### Pro rozbor MŠ

In [18]:
def get_MS_dochazka(path, sheet='1.1.9'):
    dochazka = (pd.read_excel(path, sheet_name=sheet, header=3, index_col=0, usecols='A, C, E, G, I, K')).iloc[3:-1]
    
    rok_match = re.search(r'(\d{4}-\d{2})', path)
    if rok_match:
        year = rok_match.group(1).replace('-', '/')  # Extrahovaný a upravený školní rok
    else:
        year = "Rok nebyl nalezen"
    
    dochazka.insert(loc=5, column='rok', value=year)
    return dochazka

MS_23_24 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2023-24.xlsx')
MS_22_23 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2022-23.xlsx')
MS_21_22 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2021-22.xlsx')
MS_20_21 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2020-21.xlsx')
MS_19_20 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2019-20.xlsx')
MS_18_19 = get_MS_dochazka(r'D:\git\skolstvi_projekt\CSU\MS2018-19_konkrTab.xlsx', sheet='2300421909')


MS_all = pd.concat([MS_23_24, MS_22_23, MS_21_22, MS_20_21, MS_19_20], ignore_index=False, sort=False)
MS_all
# MS_18_19

,mladší 3 let,3leté,4leté,5leté,6leté a starší,rok
Hlavní město Praha,3187,11365,12719,13562,1938,2023/24
Středočeský kraj,3707,13755,15642,16782,3071,2023/24
Jihočeský kraj,2543,5810,6452,6867,1544,2023/24
Plzeňský kraj,1565,4965,5611,6195,1160,2023/24
Karlovarský kraj,911,2220,2418,2537,316,2023/24
...,...,...,...,...,...,...
Kraj Vysočina,2421,4637,4931,4984,992,2019/20
Jihomoravský kraj,4271,11050,11898,12162,2415,2019/20
Olomoucký kraj,3436,5913,5914,6068,1600,2019/20
Zlínský kraj,2737,5154,5438,5268,1402,2019/20


### Počet cizinců ve školkách
Počet cizinců v MŠ je již zahrnut MS_all

1) údaje o fyzických osobách (každé dítě je evidováno jen pod jedním státním občanstvím, pokud má dítě dvojí občanství, upřednostní se české, dále občanství státu EU)

In [19]:
dochazka_cizinci = (pd.read_excel(r'D:\git\skolstvi_projekt\CSU\MS2023-24.xlsx', sheet_name='1.1.15', header=3, index_col=0, usecols='A, H:L')).drop(labels='Česká republika')
dochazka_cizinci

,2019/20,2020/21,2021/22,2022/23,2023/24
Hlavní město Praha,4701,4805,4893,6222,6258
Středočeský kraj,1816,1793,1891,3022,3104
Jihočeský kraj,485,470,474,959,931
Plzeňský kraj,804,807,837,1338,1324
Karlovarský kraj,489,457,394,670,612
Ústecký kraj,633,625,631,1197,1150
Liberecký kraj,479,478,441,795,755
Královéhradecký kraj,342,319,295,722,634
Pardubický kraj,370,332,332,676,632
Kraj Vysočina,222,225,211,529,577
